# __Grid Search for hyper-parameter tuning of xgboost__

Due to the somewhat erratic and therefore erratic nature eof our dataset, most likely classical time series models such as (S)ARIMA or Holt-Winters won't suffice here. Although we have identified regular time features such as seasonality, with lagging we take our time features beyond that which is outside the scope of classical time series forecast.

In the data science community, xgboost is a commonly accepted non-classical algorithm for time forecasts. Care must be taken with that model as the problem of unseen data exists, i.e. whatever target values the model has not seen in training, it cannot predict in a test set. In our case, we know that marginal CO2 emissions are within a set range for the training as well as the test set (notebook 6: Mean of CO2 emsissions of per time point).

In [7]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import joblib
import csv

import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

In [8]:
file_path = '../../big_data/df_clean_interconnectors.pkl'
df = pd.read_pickle(file_path)

In [9]:
df.head()

,CO2E_EMISSIONS_FACTOR,weekday,year,minute_sin,minute_cos,hour_sin,hour_cos,month_sin,month_cos,lag1,...,lag7,lag8,lag9,lag10,lag11,lag12,horizon0,demand,demand_capacity,interconnector
2009-09-01 00:00:00,0.986067,0,2009,0.000000,1.000000e+00,0.0,1.0,-1.0,-1.836970e-16,1.032780,...,0.938327,0.912643,0.798310,0.980592,0.473708,0.971761,0.986067,1667.12,0.603199,302.86
2009-09-01 00:05:00,0.976360,0,2009,0.500000,8.660254e-01,0.0,1.0,-1.0,-1.836970e-16,0.986067,...,0.908518,0.938327,0.912643,0.798310,0.980592,0.473708,0.976360,1657.52,0.599962,300.00
2009-09-01 00:10:00,0.976889,0,2009,0.866025,5.000000e-01,0.0,1.0,-1.0,-1.836970e-16,0.976360,...,0.971761,0.908518,0.938327,0.912643,0.798310,0.980592,0.976889,1650.15,0.596877,290.52
2009-09-01 00:15:00,1.032780,0,2009,1.000000,2.832769e-16,0.0,1.0,-1.0,-1.836970e-16,0.976889,...,0.980592,0.971761,0.908518,0.938327,0.912643,0.798310,1.032780,1630.66,0.589438,260.75
2009-09-01 00:20:00,0.975655,0,2009,0.866025,-5.000000e-01,0.0,1.0,-1.0,-1.836970e-16,1.032780,...,0.903942,0.980592,0.971761,0.908518,0.938327,0.912643,0.975655,1628.96,0.587282,256.98


### __Splitting test and validation__

Due to our lagging features, we will have to make a few conmsiderations regrading the train / validation split here.
Have a closer look onto the table below. If we want to split our dataset into a train / test split, we have to think through a few points. Hypothetically, imagine we make a clean cut at any Timestamp > 2009-07-01 05:00:00. Then, almost all values of lag1 in the train set are present in lag12 of the test set. The same goes for all remaining lag columns to a lesser extent.

This is leakage of training data into the validation set. Such leakage leads to an artificially increased predicition performance not corresponding to the performance of a completely unseem dataset.

<img src="../images/lag_examples3.jpg"> <br/>

To avoid leakage we write our own train - validation function which accounts for leakage by leaving a gap of the size of the max number of lags in between the train and validation sets.

In [11]:
def train_validation_ts(df, relative_train, maximal_lag, horizon):
    '''
    Time series (ts) split function creates a train/test set under consideration of potential overlap between the two due to lag processing
    X_train, y_train, X_test, y_test = ...
    df=must contain target column as "target"; all other columns must be used as features
    percentage_train=how much of the total dataset shall be used for training; must be added between 0 - 1
    maximal_lag=out of all lag feature engineering, enter the maximal lag number
    '''
    k = int(df.shape[0] * relative_train)
    data_train = df.iloc[:k,:]
    #to avoid overlapping of train and test data, a gap of the maximal lag - 1 must be included between the two sets
    data_test = df.iloc[k+maximal_lag:,:]
    
    assert data_train.index.max() < data_test.index.min()
    
    #returns in the sequence X_train, y_train, X_test, y_test
    return (data_train.drop(columns=[f"horizon{horizon}","CO2E_EMISSIONS_FACTOR"], axis=1), data_train[f"horizon{horizon}"],
            data_test.drop(columns=[f"horizon{horizon}","CO2E_EMISSIONS_FACTOR"], axis=1), data_test[f"horizon{horizon}"])

In [17]:
X_train, y_train, X_val, y_val = train_validation_ts(df, 0.8, 12, 0)

In [20]:
print(X_train.columns)
print(X_val.columns)

print(type(y_train))
print(type(y_val))

Index(['weekday', 'year', 'minute_sin', 'minute_cos', 'hour_sin', 'hour_cos',
       'month_sin', 'month_cos', 'lag1', 'lag2', 'lag3', 'lag4', 'lag5',
       'lag6', 'lag7', 'lag8', 'lag9', 'lag10', 'lag11', 'lag12', 'demand',
       'demand_capacity', 'interconnector'],
      dtype='object')
Index(['weekday', 'year', 'minute_sin', 'minute_cos', 'hour_sin', 'hour_cos',
       'month_sin', 'month_cos', 'lag1', 'lag2', 'lag3', 'lag4', 'lag5',
       'lag6', 'lag7', 'lag8', 'lag9', 'lag10', 'lag11', 'lag12', 'demand',
       'demand_capacity', 'interconnector'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


### __train xgboost__

below you find a set of hyperparameters which were tested for optimal forecasting results of our dataset.
The most relevant ones appeared to be learning rate, max_depth, n_estimators, and eta (not all reflected in this notebook). For the train / validation, the parameters with the best outcome were used.

In [21]:
learning_rate = [0.01, 0.1] #learning_rate
max_depth = [50]
n_estimators = [100]
reg_alpha = [0.05]
reg_lambda = [0]

params = {
    "learning_rate": learning_rate,
    "max_depth": max_depth,
    "n_estimators": n_estimators,
    "reg_alpha": reg_alpha,
    "reg_lambda": reg_lambda,
}

params

{'learning_rate': [0.01, 0.1],
 'max_depth': [50],
 'n_estimators': [100],
 'reg_alpha': [0.05],
 'reg_lambda': [0]}

## __Most importantly: time series split__

Therefore, we use the time series split function built into sklearn (tscv = TimeSeriesSplit(n_splits=n_splits).

### __K-fold cross validation__

<img src="../images/Kfold_CV.png"> <br/>
source: scikit-learn.org

### __Time series split__

<img src="../images/time_series_split.png"> <br/>
source: datascience.stackexchange.com
<br/>
<br/>
__Note: the sklearn tscv does not account for leakiness__
<br/>
<br/>

In [28]:
def XGB_GS_ts(X_train, y_train, params,run, n_splits=2, n_jobs=7, verbose=5):
    '''
    Function performs GridSearch using TimeSeries CV
    X_train, y_train
    n_splits=number of splits in TimeSeriesCV; default:3
    n_jobs=default: -1
    verbose=default:5
    '''
    
    model = xgb.XGBRegressor()

    tscv = TimeSeriesSplit(n_splits=n_splits)
    gsearch = GridSearchCV(estimator=model, cv=tscv,
                            param_grid=params, n_jobs=n_jobs, verbose=verbose)

    gsearch.fit(X_train, y_train)
    
    print("Best params were: {}".format(gsearch.best_params_))
    
    pd.DataFrame(gsearch.cv_results_).to_csv('{}/nem-data/trainings/grid_searches/{}_GS.csv'.format(os.environ['HOME'],run))
    joblib.dump(gsearch, '{}/nem-data/trainings/gridsearches/{}_GS_object.pkl'.format(os.environ['HOME'], run))
    
    best_model = gsearch.best_estimator_
    
    error_test = np.sqrt(mse(y_test, best_model.predict(X_test))/y_test.mean())
    error_train = np.sqrt(mse(y_train, best_model.predict(X_train))/y_train.mean())
    compare_train_test_error = abs(error_test - error_train)
    
    settings = {
    "Model": "XGBoost",
    "Feature Description": "sine_cosine, lag_12, horizon=0, demand, capacity, interconnectors",
    "Model Description": gsearch.best_params_
    }

    print(f"Root mean squared percentage error: {error_train, error_test}")
    log_test_results(
        settings, error_train, error_test,
        compare_train_test_error, run
    )
    
    return gsearch

In [30]:
gsearch = XGB_GS_ts(X_train,y_train,params,4)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   2 out of   4 | elapsed:  9.6min remaining:  9.6min
[Parallel(n_jobs=7)]: Done   4 out of   4 | elapsed: 19.0min remaining:    0.0s
[Parallel(n_jobs=7)]: Done   4 out of   4 | elapsed: 19.0min finished
/home/bastian/Environment/DSR_portfolio/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/bastian/Environment/DSR_portfolio/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[20:29:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best params were: {'learning_rate': 0.1, 'max_depth': 50, 'n_estimators': 100, 'reg_alpha': 0.05, 'reg_lambda': 0}
Root mean squared percentage error: (0.004771012911918365, 0.35569846008739964)


In [21]:
def log_test_results(settings, error_train, error_test, train_test_error_difference, file_name):
    csv_path = '{}/nem-data/trainings/grid_searches/{}_GS_log.csv'.format(os.environ['HOME'], file_name)
    must_add_headers = False if os.path.isfile(csv_path) else True

    with open(csv_path, mode='a') as test_results:
        writer = csv.writer(test_results,
                            delimiter=',',
                            quotechar='"',
                            quoting=csv.QUOTE_MINIMAL)

        if must_add_headers:
            writer.writerow([
                'Model', 'Feature Description', "Model Description", "Training error", "Test error",
                "Difference_train_test_error"
            ])
        writer.writerow([
            settings["Model"], settings["Feature Description"],
            str(settings["Model Description"]), error_train, error_test,
            train_test_error_difference
        ])

### __Best params__

Find the outcomes in the folder 'grid_searches'.

Note that your results might slightly vary since you may be using more NEMDE data as they are being updated.

In [23]:
max_depth = 5
learning_rate = 0.1
num_estimators = 100
reg_alpha = 0.05
reg_lambda = 0